In [10]:
import numpy as np
import requests
from numba import jit
from scipy.optimize import minimize

@jit
def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)
@jit
def total_energy(positions):
    """
    Calculate the total energy
    input:
    positions: 3*N array which represents the atomic positions
    output
    E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)

    #positions = [x0, y0, z0, x1, y1, z1, .....  , xn, yn, zn]
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            #print('pos1:  ', pos1)
            #print('pos2:  ', pos2)
            dist = np.linalg.norm(pos1-pos2)
            #print(i,j, dist)
            E += LJ(dist)
    return E
@jit           
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))
@jit
def get_pos_from_url(address='http://doye.chem.ox.ac.uk/jon/structures/LJ/points/', N=7):
    url_address = address + str(N)
    data_str = requests.get(url_address).text
    return parse_url_text(data_str)    
@jit    
def parse_url_text(data_str):
    x_array = []
    text = data_str.split('\n')
    for line in text:
        [x_array.append(float(i)) for i in line.split()]
    return np.array(x_array)

@jit
def get_min(n, m):
    f_values = []
    x_values = []
    N_attempts = 2
    N_atom = n
    for i in range(N_attempts):
        pos = init_pos(N_atom)
        res = minimize(total_energy, pos, method=m, tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
        if i%10==0:
            print('step: ', i, '  values:', res.fun)

    print('The global minimum:  ', min(f_values))
    return

In [13]:
%timeit get_min(13,'CG')

step:  0   values: -18.82867157623802
The global minimum:   -41.39439841691249
step:  0   values: -36.38045092891985
The global minimum:   -36.38045092891985
step:  0   values: -39.722599697082664
The global minimum:   -39.722599697082664
step:  0   values: -39.96537934589519
The global minimum:   -40.604584201440055
step:  0   values: -27.479738923156333
The global minimum:   -41.44459700213772
step:  0   values: -38.31768631084022
The global minimum:   -38.31768631084022
step:  0   values: -37.25947466915122
The global minimum:   -37.5646235849605
step:  0   values: -37.59447960097039
The global minimum:   -37.59447960097039
1.49 s ± 206 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit get_min(13,'TNC')

step:  0   values: -15.117727570660161
The global minimum:   -15.117727570660161
step:  0   values: -16.116091601570393
The global minimum:   -16.116091601570393
step:  0   values: -13.796318085975889
The global minimum:   -14.414463259677738
step:  0   values: -17.459217144697735
The global minimum:   -17.459217144697735
step:  0   values: -14.651458892403168
The global minimum:   -15.229574385649299
step:  0   values: -15.428976226154987
The global minimum:   -15.428976226154987
step:  0   values: -13.245881594393426
The global minimum:   -17.81209249625982
step:  0   values: -17.773250841606174
The global minimum:   -17.773250841606174
1 s ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit get_min(13,'Powell')

step:  0   values: -37.68292025372551
The global minimum:   -37.68292025372551
step:  0   values: -39.4289995631132
The global minimum:   -39.4289995631132
step:  0   values: -37.43771566431944
The global minimum:   -37.43771566431944
step:  0   values: -32.92746659892417
The global minimum:   -38.37831202173158
step:  0   values: -40.66894797612742
The global minimum:   -40.66894797612742
step:  0   values: -29.01968556239137
The global minimum:   -38.63128711785599
step:  0   values: -35.25593551025974
The global minimum:   -35.25593551025974
step:  0   values: -28.40029022093174
The global minimum:   -37.73989960729051
1.65 s ± 375 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit get_min(13,'BFGS')

step:  0   values: -37.83872214242061
The global minimum:   -40.60458420122922
step:  0   values: -36.255139294793054
The global minimum:   -38.91066818562406
step:  0   values: -38.836004003856196
The global minimum:   -38.836004003856196
step:  0   values: -37.614726446007644
The global minimum:   -39.79660913931101
step:  0   values: -9.103853540845686
The global minimum:   -39.717147417815184
step:  0   values: -41.47197984764065
The global minimum:   -41.47197984764065
step:  0   values: -39.43071596149362
The global minimum:   -39.53705533711737
step:  0   values: -41.47197984765666
The global minimum:   -41.47197984765666
789 ms ± 282 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
